In [99]:
import os
import pandas as pd
from datetime import datetime
from IPython.display import display

class PowerPlants(object):
    """
    Simplified manager for power plant data using original CSV column names.

    Methods:
        - load_new_data_from_file: read raw CSV into DataFrame.
        - save_new_data: append DataFrame rows to database CSV.
        - get_data_from_database: retrieve latest entry per plant.
        - aggregate_data_to_monthly: monthly summary of average, min, max volumes.
        - aggregate_data_to_country: total volume by country and technology.
    """
    def __init__(self, database_file: str = 'database.csv'):
        """Initialize with path to the CSV database."""
        self.database_file = database_file

    def load_new_data_from_file(self, file_path: str) -> pd.DataFrame:
        """
        Load raw data from a CSV file using original columns.

        Returns:
            DataFrame with columns: Date, Country , Technology, SiteName, Volume
        """
        return pd.read_csv(file_path)

    def save_new_data(self, input_data: pd.DataFrame, updatedby: str = 'petroineos') -> None:
        """
        Append new data to the database CSV, tagging with updater and timestamp.
        """
        df = input_data.copy()
        df['updatedby'] = updatedby
        df['updatetime'] = datetime.now()
        df.to_csv(
            self.database_file,
            mode='a',
            header=not os.path.exists(self.database_file),
            index=False
        )

    def get_data_from_database(self) -> pd.DataFrame:
        """
        Return the most recent record per plant (SiteName) based on updatetime.
        """
        df = pd.read_csv(self.database_file, parse_dates=['updatetime'])
        idx = df.groupby('SiteName')['updatetime'].idxmax()
        return df.loc[idx].reset_index(drop=True)

    def aggregate_data_to_monthly(self) -> pd.DataFrame:
        """
        Compute monthly average, minimum, and maximum volumes per plant.

        Returns:
            DataFrame with columns: month, SiteName, Technology, avg_volume, min_volume, max_volume
        """
        df = pd.read_csv(self.database_file)
        df['Date'] = pd.to_datetime(df['Date'], dayfirst=True)
        df['month'] = df['Date'].dt.to_period('M').dt.to_timestamp()
        # aggregate
        agg = df.groupby(['month', 'SiteName', 'Technology'], as_index=False)['Volume'] \
                .agg(avg_volume='mean', min_volume='min', max_volume='max')
        return agg

    def aggregate_data_to_country(self) -> pd.DataFrame:
        """
        Sum Volume by Country  and Technology.

        Returns:
            DataFrame with columns: Country , Technology, total_volume
        """
        df = pd.read_csv(self.database_file)
        return (
            df
            .groupby(['Country ', 'Technology'], as_index=False)['Volume']
            .sum()
            .rename(columns={'Volume': 'total_volume'})
        )



In [107]:
if os.path.exists('database.csv'):
    os.remove('database.csv')

pp = PowerPlants()
for f in ['wind_plants.csv', 'gas_plants.csv', 'gas_fr_plants.csv', 'database.csv']:
    data = pp.load_new_data_from_file(f)
    pp.save_new_data(data)

# Retrieve tables
latest_df  = pp.get_data_from_database()
monthly_df = pp.aggregate_data_to_monthly()
country_df = pp.aggregate_data_to_country()

# Display Latest Data
print("Latest Data per Symbol:")
display(latest_df)

# Display Pivoted Monthly Metrics
print("Monthly Metrics Pivot Table:")
pivot = (
    monthly_df
    .pivot(index='month', columns='SiteName', values=['avg_volume','min_volume','max_volume'])
)
# flatten columns
pivot.columns = [f"{site} {metric.split('_')[0].capitalize()}" for metric, site in pivot.columns]
pivot = pivot.reset_index()
display(pivot)

# Display Country Totals
print("Country Totals:")
display(country_df)

# Use the same loader to pull in the full database.csv
raw_df = pp.load_new_data_from_file(pp.database_file)

# Drop any rows missing critical fields
clean_df = raw_df.dropna(subset=['Date', 'Country ', 'Technology', 'SiteName', 'Volume'])

# Show the cleaned raw table
display(clean_df)

Latest Data per Symbol:


,Date,Country,Technology,SiteName,Volume,updatedby,updatetime
0,01/01/2024,FR,Gas,Blenod-5,6753.000000,petroineos,2025-05-30 21:52:56.670928
1,01/01/2024,GB,Wind,Hornsea-1,260.166079,petroineos,2025-05-30 21:52:56.670928
2,01/01/2024,GB,Wind,Hornsea-2,85.603428,petroineos,2025-05-30 21:52:56.670928
3,01/01/2024,GB,Gas,Pembroke-1,6570.000000,petroineos,2025-05-30 21:52:56.670928
4,01/01/2024,GB,Gas,Pembroke-2,8398.000000,petroineos,2025-05-30 21:52:56.670928


Monthly Metrics Pivot Table:


,month,Blenod-5 Avg,Hornsea-1 Avg,Hornsea-2 Avg,Pembroke-1 Avg,Pembroke-2 Avg,Blenod-5 Min,Hornsea-1 Min,Hornsea-2 Min,Pembroke-1 Min,Pembroke-2 Min,Blenod-5 Max,Hornsea-1 Max,Hornsea-2 Max,Pembroke-1 Max,Pembroke-2 Max
0,2024-01-01,5198.806452,502.454543,447.211079,7092.903226,7195.096774,3295.0,44.937929,85.603428,5390.0,5074.0,6890.0,900.648050,892.386165,8905.0,8941.0
1,2024-02-01,4889.896552,535.210353,437.902354,6738.620690,6878.551724,3489.0,9.973714,35.733954,5222.0,5051.0,6895.0,980.763621,994.618512,8970.0,8955.0
2,2024-03-01,4752.774194,432.654207,592.607677,7499.322581,6605.806452,3122.0,34.240333,122.681263,5196.0,5094.0,6399.0,864.461415,979.909946,8963.0,8685.0
3,2024-04-01,4988.466667,561.504570,504.877587,6946.566667,7142.633333,3235.0,67.202179,37.514079,5251.0,5167.0,6970.0,995.133071,999.165096,8941.0,8845.0
4,2024-05-01,5012.032258,431.650067,536.694996,7211.290323,6759.806452,3059.0,14.319806,16.694314,5009.0,5039.0,6898.0,913.805516,962.658133,8938.0,8888.0
5,2024-06-01,4928.666667,384.724509,387.852090,7003.300000,7368.933333,3083.0,16.042355,1.797161,5001.0,5247.0,6870.0,852.995514,979.952783,8978.0,8934.0
6,2024-07-01,4933.677419,471.688183,510.198669,7052.580645,7057.483871,3115.0,8.136290,13.206362,5077.0,5017.0,6680.0,944.258261,850.887204,8983.0,8981.0
7,2024-08-01,5296.258065,627.744512,474.907802,7203.032258,7455.225806,3104.0,6.695016,37.231691,5049.0,5262.0,6973.0,988.355244,999.000373,8977.0,8978.0
8,2024-09-01,4858.066667,474.785256,466.750343,7382.333333,6596.333333,3110.0,24.260159,80.138479,5149.0,5009.0,6922.0,945.447225,992.225257,8925.0,8931.0
9,2024-10-01,5365.160000,524.710537,520.673636,7202.129032,6711.483871,3061.0,75.917168,43.367492,5162.0,5101.0,6973.0,990.985647,988.350146,8923.0,8909.0


Country Totals:


,Country,Technology,total_volume
0,FR,Gas,4.759166e+06
1,GB,Gas,1.353625e+07
2,GB,Wind,9.507131e+05


,Date,Country,Technology,SiteName,Volume,updatedby,updatetime
0,01/01/2024,GB,Wind,Hornsea-1,260.166079,petroineos,2025-05-30 21:52:56.654140
1,02/01/2024,GB,Wind,Hornsea-1,709.480820,petroineos,2025-05-30 21:52:56.654140
2,03/01/2024,GB,Wind,Hornsea-1,431.527680,petroineos,2025-05-30 21:52:56.654140
3,04/01/2024,GB,Wind,Hornsea-1,223.868472,petroineos,2025-05-30 21:52:56.654140
4,05/01/2024,GB,Wind,Hornsea-1,686.985009,petroineos,2025-05-30 21:52:56.654140
...,...,...,...,...,...,...,...
5276,21/04/2025,FR,Gas,Blenod-5,3839.000000,petroineos,2025-05-30 21:52:56.670928
5277,22/04/2025,FR,Gas,Blenod-5,4461.000000,petroineos,2025-05-30 21:52:56.670928
5278,23/04/2025,FR,Gas,Blenod-5,4075.000000,petroineos,2025-05-30 21:52:56.670928
5279,24/04/2025,FR,Gas,Blenod-5,6821.000000,petroineos,2025-05-30 21:52:56.670928
